# Experiment runner for paper


For callibration run uncomment `compensation method 1 sphere` and `compensation method measurement 1 tube`

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%run paramtools.ipynb

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os.path as op
from pprint import pprint as pp
import scipy.stats
import seaborn as sns
import sys
pth = op.abspath("../")
print pth
sys.path.insert(0, pth)
import teigen
import itertools
reload(teigen)
print teigen.__path__
import teigen.tb_vtk
reload (teigen.tb_vtk)

/Users/mjirik/projects/teigen
Using QVariant and QString version 1
['/Users/mjirik/projects/teigen/teigen']


<module 'teigen.tb_vtk' from '/Users/mjirik/projects/teigen/teigen/tb_vtk.pyc'>

In [4]:
run_list = [
    # "mix 2", # test quality of compensation
    # "short",
    #"element number", # run in the morning
    #
    # == paper experiments
    #"generated volume resolution_mm", # done
    #"generated volume isotropy", # one half
    "generated volume radius", # asi ok
    # "generated volume length",  # do pulky
    #"generated volume element_number anisotropic", # asi ok
    # "generated volume noise", 
]

In [5]:
tg = teigen.tgmain.Teigen()
config = tg.get_default_config()

In [6]:
# pp(config)
config

OrderedDict([('generators',
              OrderedDict([('Voronoi tubes',
                            OrderedDict([('element_number', 30),
                                         ('radius_distribution_uniform', True),
                                         ('radius_distribution_normal', False),
                                         ('radius_distribution_fixed', False),
                                         ('radius_distribution_minimum', 2.0),
                                         ('radius_distribution_maximum', 10.0),
                                         ('radius_distribution_mean', 5.0),
                                         ('radius_distribution_standard_deviation',
                                          5.0),
                                         ('random_generator_seed', 0)])),
                           ('Gensei', OrderedDict([('n_objects', 10)])),
                           ('Continuous tubes',
                            OrderedDict([('element_number', 3

# Prepare multiple configs


In [7]:
configs = []
tg = teigen.tgmain.Teigen()

In [8]:
run_label = "mix"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47, 
        45, 53, 57]
    element_numbers = [10, 15]
    radiuses = [15., 20.]
    rnds = [#110,
            112]
    lengths = [28, 30, 32]
    radius_methods = [
                #"cylinder surface + sphere error",
                "cylinder volume + sphere error",
                # "cylinder volume + sphere error + man", # fine
                # "cylinder surface + sphere error + join error", # fine
                # "cylinder surface + sphere error + join error man" # fine
        
    ]
    for resolution, radius, rnd, radius_method, length, element_number in itertools.product(
        resolutions, radiuses, rnds, radius_methods, lengths, element_numbers):
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = element_number
                gc["radius_distribution_uniform"] = True
                gc["radius_distribution_fixed"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = radius
                gc["radius_distribution_standard_deviation"] = 0.1
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = length
                gc["length_distribution_standard_deviation"] = 10.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

In [9]:
run_label = "mix 2"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        
        45, 
        # 51, ok kromě cylinder surface + sphere error
        53, 
        55]
    element_numbers = [10]
    radiuses = [15., 20.]
    rnds = [110,
            
            112,
            113,
           ]
    lengths = [28, 30, 32]
    radius_methods = [
                # "inscribed",
                "cylinder surface + sphere error",
                "cylinder volume + sphere error",
                "cylinder surface + sphere error + join error", # fine
                "cylinder volume + sphere error + join error", # fine
                # "cylinder volume + sphere error + man", # fine
                # "cylinder surface + sphere error + join error man" # fine
        
    ]
    for resolution, radius, rnd, radius_method, length, element_number in itertools.product(
        resolutions, radiuses, rnds, radius_methods, lengths, element_numbers):
                config = tg.get_default_config()
                config["generator_id"] = "Unconnected tubes"
                config["output"]["note"] = run_label + " " + radius_method
                config["areasampling"]["voxelsize_mm"] = [1.0, 1.0, 1.0]
                config["areasampling"]["areasize_mm"] = [100.0, 100.0, 100.0]
                config["areasampling"]["areasize_px"] = [100, 100, 100]
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = element_number
                gc["radius_distribution_uniform"] = True
                gc["radius_distribution_fixed"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = radius
                gc["radius_distribution_standard_deviation"] = 0.1
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = length
                gc["length_distribution_standard_deviation"] = 10.0
                gc["random_generator_seed"] = rnd
                gc['radius_distribution_uniform'] = False
                gc['radius_distribution_normal'] = True
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

# paper data

In [10]:
## Paper figures
def get_paper_config(resolution_mm=0.02):
    config = tg.get_default_config()
    config["generator_id"] = 3
    #config["output"]["note"] = run_label + " " + radius_method
    gc = config["generators"]["Unconnected tubes"]
    gc["element_number"] = 30
    gc["radius_distribution_uniform"] = True
    gc["radius_distribution_fixed"] = False
    gc["orientation_anisotropic"] = False
    gc["radius_distribution_mean"] = 0.2
    gc["radius_distribution_standard_deviation"] = 0.1
    gc["radius_distribution_maximum"] = 3.0
    gc["radius_distribution_minimum"] = .3
    gc['radius_distribution_uniform'] = False
    gc['radius_distribution_normal'] = True
    gc["length_distribution_mean"] = 3.0
    gc["length_distribution_standard_deviation"] = .1
    gc["volume_fraction"] = 0.5
    gc["random_generator_seed"] = 100
    gc["orientation_anisotropic"] = False
    config["postprocessing"]["gaussian_blur"] = True
    config["postprocessing"]["gaussian_filter_sigma_mm"] = 0.04
    config["postprocessing"]["measurement_resolution"] = 35
    config["postprocessing"]['add_noise'] = False
    config["appearance"]["skip_volume_generation"] = False
    config["measurement"]["polygon_radius_selection_method"] = "inscribed"
    config['areasampling']["areasize_mm"] = [10.0, 10.0, 10.0]
    config['areasampling']["areasize_px"] = [10.0/resolution_mm, 10.0/resolution_mm, 10.0/resolution_mm]
    config['areasampling']["voxelsize_mm"] = [resolution_mm, resolution_mm, resolution_mm]
    # config['filepattern'] = "~/teigen_data/generated_resolution_mm_{seriesn:03d}/data{:06d}.jpg"
    return config

In [11]:
# bez šumu
run_label = "generated volume resolution_mm"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [.5, .4, .3, .2, 0.1, 0.08, 0.06, 0.04, 0.02, .01]
    element_numbers = [30]
    radiuses = [.2]
    rnds = [
        110,
       # 111,
       # 112
    ]
    lengths = [3.0]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for resolution_mm, radius_method  in itertools.product(
        resolutions_mm, radius_methods):
                config = get_paper_config(resolution_mm=resolution_mm)
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                config['filepattern'] = "~/teigen_data/teigen_resolution_mm_" + str(resolution_mm) + "_{seriesn:03d}/data{:06d}.jpg"
                configs.append(config)

In [12]:
# bez šumu
run_label = "generated volume isotropy"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [
        #.5, .4, .3, .2, 0.1, 0.08, 0.06, 
        0.04, 0.02, .01]
                       #0.4, 0.2, .1
    isotropy_values = [
        5, 4, 3, 2, 1, 0.8, 0.6, 0.3, 0.2, .1
    ]
    element_numbers = [30]
    radiuses = [.2]
    rnds = [
        110,
       # 111,
       # 112
    ]
    lengths = [3.0]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for isotropy, radius_method in itertools.product(
        isotropy_values, radius_methods):
                config = get_paper_config()
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                gc["orientation_anisotropic"] = True
                gc["orientation_variance_rad"] = isotropy
                config['filepattern'] = "~/teigen_data/teigen_isotropy_" + str(isotropy) + "_{seriesn:03d}/data{:06d}.jpg"
                configs.append(config)

In [13]:
run_label = "generated volume element_number"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [.2]
    element_numbers = [20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
    radiuses = [.5]
    rnds = [
        110,
       # 111,
       # 112
    ]
    lengths = [3.0]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for element_number, radius_method in itertools.product(
        element_numbers, radius_methods):
                config = get_paper_config()
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                gc["element_number"] = element_number
                config['filepattern'] = "~/teigen_data/teigen_element_number_" + str(element_number) + "_{seriesn:03d}/data{:06d}.jpg"
                configs.append(config)

In [14]:
run_label = "generated volume element_number anisotropic"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [.2]
    element_numbers = [20, 30, 40, 50, 60, 70, 80, 90, 100, 110]
    radiuses = [.5]
    rnds = [
        110,
       # 111,
       # 112
    ]
    lengths = [3.0]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for element_number, radius_method in itertools.product(
        element_numbers, radius_methods):
                config = get_paper_config()
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                gc["element_number"] = element_number
                gc["orientation_anisotropic"] = True
                gc["maximum_1000_iteration_number"] = 30
                config['filepattern'] = "~/teigen_data/teigen_element_number_anisotropic_" + str(element_number) + "_{seriesn:03d}/data{:06d}.jpg"
                configs.append(config)

In [15]:
run_label = "generated volume radius"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [.02]
    
    element_numbers = [30]
    radiuses = [0.06, 0.08, 0.1, 0.2, .3, .5, .8, 1.0, 1.5, 3.0]
    rnds = [
        110,
       # 111,
       # 112
    ]
    lengths = [3.0]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for radius, radius_method in itertools.product(
        radiuses, radius_methods):
                config = get_paper_config()
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                gc["radius_distribution_mean"] = radius
                gc["radius_distribution_uniform"] = False
                gc["radius_distribution_fixed"] = True
                gc["radius_distribution_normal"] = False
                config['filepattern'] = "~/teigen_data/teigen_radius_" + str(radius) + "_{seriesn:03d}/data{:06d}.jpg"
                configs.append(config)

In [16]:
run_label = "generated volume length"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [.02]
    element_numbers = [30]
    radiuses = [0.2]
    rnds = [
        110,
       # 111,
       # 112
    ]
    lengths = [
        # 0.05, 0.1, 0.2, 
        0.4, 0.6, 0.8, 1.0, 2.0, 3.0, 4.0, 
        #5.0, 6.0
    ]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for radius_method, length in itertools.product(
        radius_methods, lengths):
                config = get_paper_config()
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                # gc["radius_distribution_mean"] = radius
                gc["length_distribution_mean"] = length
                config['filepattern'] = "~/teigen_data/teigen_length_" + str(length) + "_{seriesn:03d}/data{:06d}.jpg"
                configs.append(config)

In [17]:
run_label = "generated volume noise"
if run_label in run_list:
    # if run_label in run_list:
    # for resolution in [25, 30, 35]:
    #    for rnd in [106, 107, 108, 109, 110]:
    #        for radius_method in ["best"]:
    resolutions = [
        # 26, 31, 33, 39, 
        # 41, 45, 59, 
        # 47,
        # 15,
        # 25,
        35,
        # 45,
        # 55,
        # 65,
        
        # 51, ok kromě cylinder surface + sphere error
        #53, 
        #55
    ]
    resolutions_mm = [.02]
    element_numbers = [30]
    radiuses = [.3, .5, .8, 1.1, 1.5]
    rnds = [
        #110,
        111,
        # 112
    ]
    noises = [20, 40, 60, 80, 100, 120, 140, 160, 200, 220]
    lengths = [3.0]
    radius_methods = [
                "inscribed",
                # "cylinder surface + sphere error",
                # "cylinder volume + sphere error",
                # "cylinder surface + sphere error + join error",
                # "cylinder volume + sphere error + join error", 
        
    ]
    for noise, radius_method in itertools.product( noises, radius_methods):
                config = get_paper_config()
                gc = config["generators"]["Unconnected tubes"]
                config["output"]["note"] = run_label + " " + radius_method
                config['filepattern'] = "~/teigen_data/teigen_noise_" + str(noise) + "_{seriesn:03d}/data{:06d}.jpg"
                config["postprocessing"]['add_noise'] = True
                config["postprocessing"]['noise_exponent'] = 0.
                config["postprocessing"]['noise_std'] = noise
                config["postprocessing"]['noise_mean'] = 0.
                config["postprocessing"]['noise_lambda0'] = 0.02
                config["postprocessing"]['noise_lambda1'] = 1.00
                configs.append(config)

In [18]:
run_configs(configs)

0 / 27
generated volume element_number anisotropic inscribed
rng 100, r 0.2, l 3.0, res 35
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
run finished
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
progess: 0.05
step2 finished
1 / 27
generated volume element_number anisotropic inscribed
rng 100, r 0.2, l 3.0, res 35
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
progess: 0.0333333333333
proge

KeyboardInterrupt: 